# Main

The pseudo-code of the main program is the following:

- Creates the map from `vision.py` and determines global navigation plan through A* algorithm
- Initializes position and other parameters
- Starts main loop
    - Determines if there is an unpredicted obstacle nearby
    - If so, determines control inputs to avoid obstacle, otherwise determines control inputs to reach next temporary goal
    - Determines newly reached state position with extended Kalman filter and odometry
    - Quits main loop if last goal has been reached
- Terminates program

In [ ]:
def theta_needed(p_x, p_y, n_x, n_y):
    theta = 0
    #1st quadrant
    if(n_x > p_x and n_y >= p_y):
        theta = np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
    #2nd quadrant
    elif(n_x <= p_x and n_y > p_y):
        theta = pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))
    #3rd quadrant
    elif(n_x < p_x and n_y <= p_y):
        theta = pi + np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
    #4th quadrant
    elif(n_x >= p_x and n_y < p_y):
        theta = 3*pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))
    
    return theta

In [ ]:
#------MAIN SANS OBSTACLE AVOIDANCE AVEC ODOMETRY------
occupancy_grid, list_paintings_grid, start_grid, goal_grid, fig, ax= create_fill_grid()
occupancy_grid = checkerboard(occupancy_grid)
path = thymio_path(list_paintings_grid, start_grid, goal_grid, fig, ax, occupancy_grid)

start_cm = convert_grid_to_cm(start_grid[0],start_grid[1])
Pablo = Thymio(start_cm[0], start_cm[1], 100)

Threshold_theta = 0.001 # Threshold in radians
Threshold_d = 0.2 # Threshold in cm
current_goal = path.remove(path[0])
current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
p_x = Pablo.pos[0]
p_y = Pablo.pos[1]
VelLeft = 0
VelRight = 0
adjust = 1.9 #4.15
b = 0.095 # distance between Thymio's wheels in m



#Loop tant qu'on est pas au goal final
while((Pablo.pos[0]<path[-1][0]-Threshold_d or Pablo.pos[0]>path[-1][0] +Threshold_d) and
      (Pablo.pos[1]<path[-1][1]-Threshold_d or Pablo.pos[1]>path[-1][1] +Threshold_d)):
    th.set_var("motor.right.target", 0)
    th.set_var("motor.left.target", 0)
    a = time.time();
    t = b-a
    SpeedGain = adjust*0.0325 * t

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Pablo.set_pos(Pablo.pos[0]+DS*SpeedGain*math.sin(math.radians(Theta)), Pablo.pos[1] + DS*SpeedGain*math.cos(math.radians(Theta)) , Pablo.pos[2]+DTheta)
    
    #Correction orientation en temps réel si pas arrivé
    if ((Pablo.pos[0] < current_goal[0] - Threshold_d or Pablo.pos[0] > current_goal[0] + Threshold_d) or
        (Pablo.pos[1] < current_goal[1] - Threshold_d or Pablo.pos[1] > current_goal[1] + Threshold_d):
        theta = theta_needed(p_x, p_y, Pablo.pos[0], Pablo.pos[1])
        p_x = Pablo.pos[0]
        p_y = Pablo.pos[1]
        if(Pablo.pos[2] < theta - Threshold_theta):
            th.set_var("motor.right.target", 100)
            th.set_var("motor.left.target", 2**16 - 100)
            b = time.time()
            VelRight = 100
            VelLeft = -100
        elif(Pablo.pos[2] > theta + Threshold_theta):
            th.set_var("motor.right.target", 2**16 - 100)
            th.set_var("motor.left.target", 100)
            b = time.time()
            VelRight = -100
            VelLeft = 100
        else:
            th.set_var("motor.right.target", 100)
            th.set_var("motor.left.target", 100)
            b = time.time()
            VelRight = 100
            VelLeft = 100

    #Si arrivé
    elif (Pablo.pos[0] >= current_goal[0] - Threshold_d and Pablo.pos[0] <= current_goal[0] + Threshold_d and
          Pablo.pos[1] >= current_goal[1] - Threshold_d and Pablo.pos[1] <= current_goal[1] + Threshold_d):
            #On vide la liste au fur et à mesure qu'on fait les déplacements + conversion en cm
            current_goal = path.remove(path[0])
            current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
            b = time.time()
            VelRight = 0
            VelLeft = 0

            #Si on chope la même pos, on prend la suivante
            while (Pablo.pos[0] >= current_goal[0] - Threshold_d and Pablo.pos[0] <= current_goal[0] + Threshold_d and
                   Pablo.pos[1] >= current_goal[1] - Threshold_d and Pablo.pos[1] <= current_goal[1] + Threshold_d):
                current_goal = path.remove(path[0])
                current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
                b = time.time()
                VelRight = 0
                VelLeft = 0
    time.sleep(1 * 10**(-3)) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

In [ ]:
#------MAIN AVEC OBSTACLE AVOIDANCE ET ODOMETRY------
occupancy_grid, list_paintings_grid, start_grid, goal_grid, fig, ax= create_fill_grid()
occupancy_grid = checkerboard(occupancy_grid)
path = thymio_path(list_paintings_grid, start_grid, goal_grid, fig, ax, occupancy_grid)

start_cm = convert_grid_to_cm(start_grid[0],start_grid[1])
Pablo = Thymio(start_cm[0], start_cm[1], 100)

State = 0 # State == 0 => Thymio goes straight, State == 1 => Thymio is in obstacle avoidance mode
Obstacle_gain = [10, 15]
Threshold_enter_obstavoid = 20 # Threshold to enter obstacle avoidance mode
Threshold_quit_obstavoid = 10 # Threshold to quit obstacle avoidance mode
Threshold_theta = 0.001 # Threshold in radians
Threshold_d = 0.2 # Threshold in cm
current_goal = path.remove(path[0])
current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
p_x = Pablo.pos[0]
p_y = Pablo.pos[1]
VelLeft = 0
VelRight = 0
adjust = 1.9 #4.15
b = 0.095 # distance between Thymio's wheels in m



#Loop tant qu'on est pas au goal final
while((Pablo.pos[0]<path[-1][0]-Threshold_d or Pablo.pos[0]>path[-1][0] +Threshold_d) and
      (Pablo.pos[1]<path[-1][1]-Threshold_d or Pablo.pos[1]>path[-1][1] +Threshold_d)):
    th.set_var("motor.right.target", 0)
    th.set_var("motor.left.target", 0)
    a = time.time();
    t = b-a
    SpeedGain = adjust*0.0325 * t

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Pablo.set_pos(Pablo.pos[0]+DS*SpeedGain*math.sin(math.radians(Theta)), Pablo.pos[1] + DS*SpeedGain*math.cos(math.radians(Theta)) , Pablo.pos[2]+DTheta)
    
    if (State == 0 and
        (th["prox.horizontal"][0] > Threshold_enter_obstavoid or
        th["prox.horizontal"][1] > Threshold_enter_obstavoid or
        th["prox.horizontal"][2] > Threshold_enter_obstavoid or
        th["prox.horizontal"][3] > Threshold_enter_obstavoid or
        th["prox.horizontal"][4] > Threshold_enter_obstavoid)): # enter obstacle avoidance
        State = 1
    elif (State == 1 and
        th["prox.horizontal"][0] < Threshold_quit_obstavoid and
        th["prox.horizontal"][1] < Threshold_enter_obstavoid and
        th["prox.horizontal"][2] < Threshold_enter_obstavoid and
        th["prox.horizontal"][3] < Threshold_enter_obstavoid and
        th["prox.horizontal"][4] < Threshold_quit_obstavoid): # quit obstacle avoidance
        State = 0
    
    if State == 0: #not avoidance, we do whatever        
        #Correction orientation en temps réel si pas arrivé
        if ((Pablo.pos[0] < current_goal[0] - Threshold_d or Pablo.pos[0] > current_goal[0] + Threshold_d) or
            (Pablo.pos[1] < current_goal[1] - Threshold_d or Pablo.pos[1] > current_goal[1] + Threshold_d):
            theta = theta_needed(p_x, p_y, Pablo.pos[0], Pablo.pos[1])
            p_x = Pablo.pos[0]
            p_y = Pablo.pos[1]
            if(Pablo.pos[2] < theta - Threshold_theta):
                th.set_var("motor.right.target", 100)
                th.set_var("motor.left.target", 2**16 - 100)
                b = time.time()
                VelRight = 100
                VelLeft = -100
            elif(Pablo.pos[2] > theta + Threshold_theta):
                th.set_var("motor.right.target", 2**16 - 100)
                th.set_var("motor.left.target", 100)
                b = time.time()
                VelRight = -100
                VelLeft = 100
            else:
                th.set_var("motor.right.target", 100)
                th.set_var("motor.left.target", 100)
                b = time.time()
                VelRight = 100
                VelLeft = 100
            
        #Si arrivé
        elif (Pablo.pos[0] >= current_goal[0] - Threshold_d and Pablo.pos[0] <= current_goal[0] + Threshold_d and
              Pablo.pos[1] >= current_goal[1] - Threshold_d and Pablo.pos[1] <= current_goal[1] + Threshold_d):
                #On vide la liste au fur et à mesure qu'on fait les déplacements + conversion en cm
                current_goal = path.remove(path[0])
                current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
                b = time.time()
                VelRight = 0
                VelLeft = 0
                
                #Si on chope la même pos, on prend la suivante
                while (Pablo.pos[0] >= current_goal[0] - Threshold_d and Pablo.pos[0] <= current_goal[0] + Threshold_d and
                       Pablo.pos[1] >= current_goal[1] - Threshold_d and Pablo.pos[1] <= current_goal[1] + Threshold_d):
                    current_goal = path.remove(path[0])
                    current_goal = convert_grid_to_cm(current_goal[0],current_goal[1])
                    b = time.time()
                    VelRight = 0
                    VelLeft = 0
            
    else: # avoid obstacle depending on sensors values
        if (th["prox.horizontal"][0] == 0 and
            th["prox.horizontal"][1] == 0 and
            th["prox.horizontal"][3] == 0 and
            th["prox.horizontal"][4] == 0):
            Left_gain = 0
            Right_gain = int(Obstacle_gain[1]*(th["prox.horizontal"][2]/100))
        else:
            Left_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][4]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][3]/100))
            Right_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][0]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][1]/100))
        
        if Left_gain > 100:
            th.set_var("motor.left.target", 2**16 + (100 - Left_gain))
            VelLeft = -100 + Left_gain
        else:
            th.set_var("motor.left.target", (100 - Left_gain))
            VelLeft = 100 - Left_gain
        
        if Right_gain > 100:
            th.set_var("motor.right.target", 2**16 + (100 - Right_gain))
            VelRight = -100 + Right_gain
        else:
            th.set_var("motor.right.target", (100 - Right_gain))
            VelRight = 100 - Right_gain
        b = time.time()
        
    time.sleep(1 * 10**(-3)) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)